<a href="https://colab.research.google.com/github/astha-jha/Code_Smell_Remover/blob/main/DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source code modelling using LSTM



In [ ]:
!mkdir -p Dataset
!google-drive-ocamlfuse drive

/bin/bash: line 1: google-drive-ocamlfuse: command not found


In [ ]:

!pip install torch
!pip install torchvision

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load in Data



In [ ]:
with open('/content/drive/MyDrive/Vectorization/dictionary.csv', 'r') as f:
    text = f.read()
    dictionary = text.split('\n')
    dictionary = list(filter(None, dictionary))

with open('/content/drive/MyDrive/Vectorization/code_corpus.csv', 'r') as f:
    text = f.read()
    methods = text.split('\n')

    methods = list(filter(None, methods))
    for i in range(len(methods)):
      methods[i] = ','.join(list(filter(None, methods[i].split(','))))

dictionary.append('<unk>')
dictionary.append('<num>')

### Encoding using dictionary


In [ ]:
int2token

{0: 'getLocation',
 1: 'getEStructuralFeatures',
 2: 'ArrayList',
 3: 'FileNotFoundException',
 4: 'channelManager',
 5: 'String',
 6: 'salary',
 7: 'ruleCompanyPositionsAssignment5',
 8: 'ruleModelCompaniesAssignment2',
 9: 'score',
 10: 'quote',
 11: 'music',
 12: 'entryRuleManager',
 13: 'protected',
 14: 'optString',
 15: 'ResultSet',
 16: 'isManager',
 17: 'depTemp',
 18: 'ruleModelCompaniesAssignment3',
 19: '!',
 20: '"',
 21: 'ruleDepartmentGroup3Impl',
 22: 'printStackTrace',
 23: '%',
 24: "'",
 25: '(',
 26: 'loop2',
 27: ')',
 28: 'loop3',
 29: '*',
 30: 'google',
 31: 'DEPARTMENT',
 32: '+',
 33: 'ray',
 34: 'loop1',
 35: '-',
 36: 'ZoneXZ',
 37: 'getEmployeeImplRule',
 38: '.',
 39: 'getLeftCurlyBracketKeyword2',
 40: 'Employee',
 41: 'size',
 42: 'loop6',
 43: 'left',
 44: 'loop4',
 45: 'getPositionsKeyword3',
 46: 'loop5',
 47: 'testValidity',
 48: ':',
 49: ';',
 50: '<',
 51: '=',
 52: '>',
 53: 'object',
 54: '?',
 55: 'turnRear',
 56: 'FOLLOWEOFinentryRuleEInt488',


In [ ]:
int2token = dict(enumerate(dictionary))
token2int = {ch: ii for ii, ch in int2token.items()}

#encoded = np.array([token2int[ch] for ch in text])

In [ ]:
token2int

{'getLocation': 0,
 'getEStructuralFeatures': 1,
 'ArrayList': 2,
 'FileNotFoundException': 3,
 'channelManager': 4,
 'String': 5,
 'salary': 6,
 'ruleCompanyPositionsAssignment5': 7,
 'ruleModelCompaniesAssignment2': 8,
 'score': 9,
 'quote': 10,
 'music': 11,
 'entryRuleManager': 12,
 'protected': 13,
 'optString': 14,
 'ResultSet': 15,
 'isManager': 16,
 'depTemp': 17,
 'ruleModelCompaniesAssignment3': 18,
 '!': 19,
 '"': 20,
 'ruleDepartmentGroup3Impl': 21,
 'printStackTrace': 22,
 '%': 23,
 "'": 24,
 '(': 25,
 'loop2': 26,
 ')': 27,
 'loop3': 28,
 '*': 29,
 'google': 30,
 'DEPARTMENT': 31,
 '+': 32,
 'ray': 33,
 'loop1': 34,
 '-': 35,
 'ZoneXZ': 36,
 'getEmployeeImplRule': 37,
 '.': 38,
 'getLeftCurlyBracketKeyword2': 39,
 'Employee': 40,
 'size': 41,
 'loop6': 42,
 'left': 43,
 'loop4': 44,
 'getPositionsKeyword3': 45,
 'loop5': 46,
 'testValidity': 47,
 ':': 48,
 ';': 49,
 '<': 50,
 '=': 51,
 '>': 52,
 'object': 53,
 '?': 54,
 'turnRear': 55,
 'FOLLOWEOFinentryRuleEInt488': 56,


In [ ]:
methods = methods[:-1]

In [ ]:
encoded_methods = [[token2int[el] for el in method.split(',')] for method in methods]

In [ ]:
# token2int['<unk>']

In [ ]:
unknown_token = token2int['<unk>']

batch_size = 50
sequence_length = 100

In [ ]:
len(encoded_methods[0])

1280

In [ ]:
encoded_methods_all = []
for i, el in enumerate(encoded_methods):
  if len(el) < sequence_length:
    encoded_methods[i].extend([unknown_token] * (sequence_length - len(el)))

  enough = int(len(encoded_methods[i]) / sequence_length)
  encoded_methods_all.extend(encoded_methods[i][:enough * sequence_length])

print(len(encoded_methods_all))

print(encoded_methods_all[:5])




4311700
[977, 5, 883, 25, 27]


In [ ]:
def get_batches(arr, batch_size, seq_length):

    n_batches = int(np.floor(arr.size / (batch_size * seq_length)))

    arr = arr[:n_batches * (batch_size * seq_length)]
    arr = arr.reshape(batch_size, -1)

    for n in range(0, arr.shape[1], seq_length):

        x = arr[:,n:n+seq_length]

        if n+seq_length == arr.shape[1]:
          y = np.zeros_like(x)
          y[:,:seq_length - 1] = arr[:,n+1:n+seq_length]
          y[:,seq_length - 1] = arr[:,0]
        else:
          y = arr[:,n+1:n+seq_length+1]
        yield x, y

In [ ]:
em = np.array(encoded_methods_all)

batches = get_batches(em, 50, 100)
x, y = next(batches)

### LSTM Neural Network Definition


In [ ]:

train_on_gpu =  True
if(train_on_gpu):
    print('Training on GPU!')
else:
    print('No GPU available, training on CPU;   n_epochs very small.')

Training on GPU!


In [ ]:
class SCRNN(nn.Module):

    def __init__(self, tokens, embedding_dim = 50, n_hidden=50, n_layers=2,
                               drop_prob=0.7, lr=0.001, dictionary_size=5001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        self.embedding = nn.Embedding(len(tokens), embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc = nn.Linear(n_hidden, len(tokens))



    def forward(self, x, hidden):

        embedding = self.embedding(x)

        out_r, hidden = self.lstm(embedding, hidden)

        out_features = out_r.reshape([out_r.shape[0] * out_r.shape[1], self.n_hidden])

        out = self.dropout(out_r)

        out = out.reshape([out.shape[0] * out.shape[1], self.n_hidden])

        out = self.fc(out)

        return out, hidden, out_features


    def init_hidden(self, batch_size):

        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden


In [ ]:
def train(net, train_data, val_data, epochs=10, batch_size=batch_size, seq_length=sequence_length, lr=0.001, clip=5, print_every=10):

    best_loss = 100000

    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    if(train_on_gpu):
        net.cuda()

    counter = 0
    for e in range(epochs):

        h = net.init_hidden(batch_size)

        for x, y in get_batches(train_data, batch_size, seq_length):

            counter += 1

            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()

            output, h, _ = net(inputs, h)

            loss = criterion(output, targets.contiguous().view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):

                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h, _ = net(inputs, val_h)

                    val_loss = criterion(output, targets.contiguous().view(batch_size*seq_length))

                    val_losses.append(val_loss.item())

                net.train()

                val_loss = np.mean(val_losses)
                if val_loss < best_loss:
                  save_model(net)
                  best_loss = val_loss
                  print('Persisted the best model.')

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(val_loss),
                      "Perplexity: {:.4f}".format(np.exp(val_loss)))

In [ ]:

n_hidden= 512
n_layers= 2
embedding_dim = 50

net = SCRNN(dictionary, embedding_dim, n_hidden, n_layers)
print(net)

print('Number of parameters: ')
np.sum([np.prod(x.shape) for x in net.parameters()])

SCRNN(
  (embedding): Embedding(1001, 50)
  (lstm): LSTM(50, 512, num_layers=2, batch_first=True, dropout=0.7)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): Linear(in_features=512, out_features=1001, bias=True)
)
Number of parameters: 


3819883

In [ ]:
def save_model(net):
  model_name = '/content/Dataset/SCRNN.net'

  checkpoint = {'n_hidden': net.n_hidden,
                'n_layers': net.n_layers,
                'state_dict': net.state_dict(),
                'tokens': dictionary,
                'int2token': int2token,
                'token2int': token2int}

  with open(model_name, 'wb') as f:
      torch.save(checkpoint, f)

In [ ]:
import numpy as np

val_frac = 0.1

encoded_methods_array = np.array(encoded_methods_all)

seqs = int(encoded_methods_array.shape[0] / sequence_length)

sep_point = int(seqs * (1 - val_frac)) * sequence_length
train_methods, val_methods = encoded_methods_array[:sep_point], encoded_methods_array[sep_point:]

unknown_token_id = token2int['<unk>']

train_tokens = set(train_methods)
val_tokens = set(val_methods)

print(len(train_tokens))
print(len(val_tokens))

unknown_tokens = val_tokens - train_tokens

print(len(unknown_tokens))

val_methods = np.array([token if token not in unknown_tokens else unknown_token_id for token in val_methods])

print(len(set(val_methods)))

937
284
63
221


In [ ]:
CUDA_LAUNCH_BLOCKING=1
n_epochs = 3

train(net, train_methods, val_methods, epochs=n_epochs, batch_size=batch_size, seq_length=sequence_length, lr=0.001, print_every=50)

Persisted the best model.
Epoch: 1/3... Step: 50... Loss: 0.5132... Val Loss: 2.4067 Perplexity: 11.0968
Epoch: 1/3... Step: 100... Loss: 0.4248... Val Loss: 2.5695 Perplexity: 13.0595
Epoch: 1/3... Step: 150... Loss: 0.4229... Val Loss: 2.5244 Perplexity: 12.4834
Epoch: 1/3... Step: 200... Loss: 0.4672... Val Loss: 2.4747 Perplexity: 11.8782
Epoch: 1/3... Step: 250... Loss: 0.4357... Val Loss: 2.5307 Perplexity: 12.5626
Epoch: 1/3... Step: 300... Loss: 0.5059... Val Loss: 2.4783 Perplexity: 11.9210
Epoch: 1/3... Step: 350... Loss: 0.3973... Val Loss: 2.5664 Perplexity: 13.0190
Epoch: 1/3... Step: 400... Loss: 0.4016... Val Loss: 2.5540 Perplexity: 12.8585
Epoch: 1/3... Step: 450... Loss: 0.4037... Val Loss: 2.4403 Perplexity: 11.4765
Epoch: 1/3... Step: 500... Loss: 0.4087... Val Loss: 2.5981 Perplexity: 13.4379
Epoch: 1/3... Step: 550... Loss: 0.3797... Val Loss: 2.5779 Perplexity: 13.1701
Epoch: 1/3... Step: 600... Loss: 0.4650... Val Loss: 2.5283 Perplexity: 12.5320
Epoch: 1/3... S

In [ ]:
# import torch
# model_name = '/content/Dataset/SCRNN.net'

# checkpoint = {}

# with open(model_name, 'wb') as f:
#     torch.save(checkpoint, f)

In [ ]:
def predict(token2int, int2token, net, token, h=None, top_k=None):

        x = np.array([[token2int[token]]])

        inputs = torch.from_numpy(x)

        # detach hidden state from history
        h = tuple([each.data for each in h])

        out, h, out_features = net(inputs, h)

        p = F.softmax(out, dim=1).data

        p, top_tokens = p.topk(top_k)
        top_tokens = top_tokens.numpy().squeeze()

        p = p.numpy().squeeze()
        token = np.random.choice(top_tokens, p=p/p.sum())

        return int2token[token], h, out_features

In [ ]:
def sample(token2int, int2token, net, size, prime=[], top_k=5):

    net.cpu()
    net.eval() # eval mode

    h = net.init_hidden(1)

    features = []
    tokens = prime
    for token in prime:
        token, h, out_features = predict(token2int, int2token, net, token, h, top_k=top_k)
        features.append(out_features)

    if size > 0:
      tokens.append(token)

      for ii in range(size):
          token, h, out_features = predict(token2int, int2token, net, tokens[-1], h, top_k=top_k)
          tokens.append(token)
          features.append(out_features)

    return ''.join(tokens), features

## Loading a checkpoint

In [ ]:
with open('/content/drive/MyDrive/Dataset/SCRNN.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = SCRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
train_on_gpu = False

code, features = sample(checkpoint['token2int'], checkpoint['int2token'], loaded, 0, top_k=5, prime=['while', '('])

print(features[0].shape)
print(features[1].shape)


torch.Size([1, 512])
torch.Size([1, 512])


In [ ]:
''.join([int2token[x] for x in val_methods[510001:510100]])

''

In [ ]:
[int2token[x] for x in val_methods[510001:510010]]

[]

**Feature extraction**

In [ ]:
path = '/content/drive/MyDrive/Dataset/smells-dataset'
target = '/content/drive/MyDrive/Dataset/features_lstm.csv'
unknown_token = '<unk>'
features_num = 512
print_every = 5

with open('/content/Dataset/SCRNN.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = SCRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

i = 0
_, dirnames, _ = next(os.walk(path))

output_dataset = pd.DataFrame(index=range(len(dirnames)), columns=list(range(features_num)) + ['sevrity', 'smell', 'id'])
for dir in dirnames:
  parts = dir.split('_')
  smell = parts[0]
  severity = parts[1]
  id = parts[2]

  subpath = os.path.join(path, dir)
  _, _, filenames = next(os.walk(subpath))

  methods_features = np.zeros((1,features_num))
  for file in filenames:
    file_path = os.path.join(subpath, file)
    with open(file_path, 'r') as f:
      content = f.read()
      content_tokens = content.split(',')
      content_tokens = [x.replace('\n', '') if x.replace('\n','') in checkpoint['tokens'] else unknown_token for x in content_tokens]

      code, features = sample(checkpoint['token2int'], checkpoint['int2token'], loaded, 0, prime=content_tokens)

      method_features = np.mean(list(map(lambda x: x.detach().numpy(), features)), axis = 0)
      methods_features += method_features

  methods_features /= len(filenames)
  output_dataset.iloc[i] = np.concatenate((methods_features, np.array([severity, smell, id]).reshape(1,-1)), axis = 1)

  i += 1
  if i % print_every == 0:
    print(f'Processed {i}/{len(dirnames)}')

output = output_dataset.to_csv()
with open(target, "w") as f:
  f.write(output)



StopIteration: ignored